In [1]:
%pylab qt5

Populating the interactive namespace from numpy and matplotlib


In [2]:
from drivers.Agilent_PNA_L import *
from drivers.Yokogawa_GS200 import *
from drivers.KeysightAWG import *
from drivers.E8257D import MXG

In [3]:
from lib2.MeasurementResult import MeasurementResult

In [4]:
vna = Agilent_PNA_L("PNA-L2")
current_src = Yokogawa_GS210("GS210_3")
current_src.set_current_limits(-12e-3, 12e-3)
ro_awg = KeysightAWG("AWG3")
q_awg = KeysightAWG("AWG2")
mxg = MXG("MXG")

In [13]:
from importlib import reload
import lib2

In [6]:
# vna._visainstrument.write("CALC:PAR:DEL:ALL")
# vna._visainstrument.write("CALC:PAR:DEF:EXT 'CH1_S43_1',S43")
# vna._visainstrument.write("CALC:PAR:SEL 'CH1_S43_1'")

## Single-tone spectroscopy

In [29]:
# reload(lib2.SingleToneSpectroscopy)
from lib2.SingleToneSpectroscopy import *

#### No mixer 

In [17]:
STS = SingleToneSpectroscopy("VI-anticrossing", "Xmon Al BMSTU S444 2", vna, "current", current.set_current, 0)
vna_parameters = {"bandwidth":250, "freq_limits":(7030e6, 7070e6), "nop":101, "power":-40, "averages":1}
STS.setup_control_parameters(vna_parameters, linspace(1.6e-3, 3.3e-3, 100))
result = STS.launch()
result.visualize()

Started at:  Fri Apr 21 16:23:38 2017
Time left: 0 h 0 m 46.14 s, parameter value: 1.600e-03, average cycle time: 0.47 s          

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 25.07 s, parameter value: 2.253e-03, average cycle time: 0.41 s          

In [10]:
result.visualize()

In [18]:
result.save()

#### Through mixer

In [23]:
ro_awg.output_continuous_wave(0,0,0,5)

In [31]:
STS = SingleToneSpectroscopy("VI-anticrossing-through-mixer", "Xmon Al BMSTU S444 2", vna, "current", current.set_current, 0)
vna_parameters = {"bandwidth":250, "freq_limits":(7040e6, 7060e6), "nop":101, "power":-30, "averages":1}
currents = linspace(1.6e-3, 3.1e-3, 100)
STS.setup_control_parameters(vna_parameters, currents)
result = STS.launch()
result.visualize()

Started at:  Sat Apr 22 00:43:20 2017
Time left: 0 h 0 m 44.65 s, parameter value: 1.600e-03, average cycle time: 0.45 s          

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s, parameter value: 3.100e-03, average cycle time: 0.41 s            

## Two-tone spectroscopy

In [70]:
reload(lib2.TwoToneSpectroscopyBase)
reload(lib2.TwoToneSpectroscopy)
from lib2.TwoToneSpectroscopy import *

In [5]:
q_awg.output_continuous_wave(0,0,0,5)

##### Qubit spectrum

In [6]:
mxg.set_output_state("OFF")
TTS = FluxTwoToneSpectroscopy("VI-2tone", "Xmon Al BMSTU S444 2", vna, mxg, current_src.set_current)
vna_parameters = {"bandwidth":20, "freq_limits":(7040e6, 7052.5e6), "nop":1, "power":-40, "averages":1}
mw_src_parameters = {"power":-5, "frequency":None}
mw_src_frequencies = linspace(7e9, 8.5e9, 100)
currents = linspace(2.0e-3, 2.6e-3, 25)
TTS.setup_control_parameters(vna_parameters, mw_src_parameters, mw_src_frequencies, currents)

Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    at qubit's sweet spot (2.30 mA)


KeyboardInterrupt: 

In [28]:
mxg.set_output_state("ON")
result = TTS.launch()

Started at:  Sat Apr 22 12:39:45 2017
Time left: 5 h 42 m 48.96 s, parameter value: 2.000e-03, average cycle time: 8.24 s            

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 2 m 21.03 s, parameter value: 2.350e-03, average cycle time: 0.14 s           

In [25]:
result.visualize()

In [37]:
result.save()

##### Linewidth from excitation power

In [75]:
mxg.set_output_state("OFF")
PTS = PowerTwoToneSpectroscopy("VI-powerscan", "Xmon Al BMSTU S444 2", vna, mxg, current_src.set_current, mxg.set_power)
vna_parameters = {"bandwidth":1, "freq_limits":(7040e6, 7052.5e6), "nop":1, "power":-40, "averages":5}
mw_src_parameters = {"power":-1, "frequency":-1}
mw_src_frequencies = linspace(7.88e9, 7.92e9,200)
mw_src_powers = linspace(-50, -10, 41)
current = 2.3e-3
PTS.setup_control_parameters(vna_parameters, mw_src_parameters, mw_src_frequencies, mw_src_powers, current)
PTS._measurement_result.set_phase_units("deg")

Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    at current of 2.30 mA
Detected frequency is 7.04707 GHz, at 2.34 mU and 36.81 degrees


In [ ]:
mxg.set_output_state("ON")
result = PTS.launch()

Started at:  Sun Apr 23 00:51:46 2017


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 53 m 0.23 s, parameter value: -1.300e+01, average cycle time: 4.54 s              

In [48]:
vna.set_parameters(PTS._pre_measurement_vna_parameters)

In [46]:
result.visualize()

In [20]:
result.save()

In [63]:
mxg.set_frequency(7.898e9)
mxg.set_power(-30)
mxg.set_output_state("OFF")
vna.prepare_for_stb(); vna.sweep_single(); vna.wait_for_stb()
ground_state_sdata = vna.get_sdata()

mxg.set_output_state("ON")
vna.prepare_for_stb(); vna.sweep_single(); vna.wait_for_stb()
mixed_state_sdata = vna.get_sdata()

In [64]:
plot(abs(ground_state_sdata))
plot(abs(mixed_state_sdata))
plot((abs(ground_state_sdata)[10:]+abs(ground_state_sdata)[:-10])/2)

##### AC Stark

In [30]:
mxg.set_output_state("OFF")
ASTS = AcStarkTwoToneSpectroscopy("VI-ac-stark", "Xmon Al BMSTU S444 2", vna, mxg, current_src.set_current, mxg.set_power)
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":1, "power":None, "averages":1}
mw_src_parameters = {"power":-30, "frequency":None}
mw_src_frequencies = linspace(7.88e9, 7.9e9, 100)
vna_powers = linspace(-50, -20, 31)
current = 2.3e-3
ASTS.setup_control_parameters(vna_parameters, mw_src_parameters, mw_src_frequencies, vna_powers, current)

Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    at current of 2.30 mA


Detected frequency is 7.04710 GHz, at 2.43 mU and 41.46 degrees


In [31]:
mxg.set_output_state("ON")
result = ASTS.launch()

Started at:  Sat Apr 22 22:57:48 2017
Time left: 25 h 28 m 58.39 s, parameter value: -5.000e+01, average cycle time: 29.62 s          

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s, parameter value: -2.000e+01, average cycle time: 0.17 s               

In [37]:
result.visualize()

In [34]:
result.save()

## Loading

In [5]:
result = MeasurementResult.load("Xmon Al BMSTU S444 2", "VI-powerscan")

0 data/Xmon Al BMSTU S444 2\Apr 22 2017\12-49-52 - VI-powerscan\VI-powerscan.pkl
1 data/Xmon Al BMSTU S444 2\Apr 22 2017\13-23-30 - VI-powerscan\VI-powerscan.pkl
2 data/Xmon Al BMSTU S444 2\Apr 22 2017\18-39-20 - VI-powerscan\VI-powerscan.pkl
More than one file found. Enter an index from listed above:
2


In [7]:
result.set_phase_units("deg")

In [8]:
result.visualize()